In [9]:
%load_ext autoreload
%autoreload 2
import os
import json
from tqdm import tqdm
import torch 
import torch.nn as nn
import models
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import Adam
from torchvision import datasets, transforms

print(f"device avaible : {'cuda' if torch.cuda.is_available() else 'cpu'}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
device avaible : cpu


# **Dataset**

Création de deux datasets distincts : 
- A contient les images des classes de 0 à 4
- B contient les images des classes de 5 à 9 

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


train_set, val_set = train_test_split(list(range(len(cifar10))), test_size=0.2, random_state=42)

def create_subset(dataset, indices, class_range):
    subset_indices = [i for i in indices if dataset.targets[i] in class_range]
    return Subset(dataset, subset_indices)

train_set_A = create_subset(cifar10, train_set, range(0, 5))
train_set_B = create_subset(cifar10, train_set, range(5, 10))

val_set_A = create_subset(cifar10, val_set, range(0, 5))
val_set_B = create_subset(cifar10, val_set, range(5, 10))


train_loader_A = DataLoader(train_set_A, batch_size=64, shuffle=True)
train_loader_B = DataLoader(train_set_B, batch_size=64, shuffle=True)

val_loader_A = DataLoader(val_set_A, batch_size=64, shuffle=False)
val_loader_B = DataLoader(val_set_B, batch_size=64, shuffle=False)

test_loader_A = DataLoader(create_subset(cifar10_test, range(len(cifar10_test)), range(0, 5)), batch_size=64, shuffle=False)
test_loader_B = DataLoader(create_subset(cifar10_test, range(len(cifar10_test)), range(5, 10)), batch_size=64, shuffle=False)


100%|██████████| 170498071/170498071 [00:10<00:00, 16511541.14it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [15]:
loss_fn = nn.CrossEntropyLoss()
path = "./resultats_cifar/"

resA  = []
resB = []
print("Entrainement du modèle A")
print()
for _ in range(10): 
    modelA = models.CNN_cifar()
    optimizerA = Adam(modelA.parameters())
    modelA.train_model(train_loader_A,test_loader_A, loss_fn, n_epochs=2, optimizer=optimizerA)
    resA.append(modelA.test(test_loader_A))
torch.save(modelA, "./models_cifar/baseA.torch")

print()
print("Entrainement du modèle B")
print()
for _ in range(10): 
    modelB = models.CNN_cifar()
    optimizerB= Adam(modelB.parameters())
    modelB.train_model(train_loader_B,test_loader_B, loss_fn, n_epochs=2, optimizer=optimizerB)
    resB.append(modelB.test(test_loader_B))
torch.save(modelB, "./models_cifar/baseB.torch")

with open(os.path.join(path, "base.json"), 'a') as file:
        json.dump({"model A" : resA}, file)
        json.dump({"model B" : resB}, file)

Entrainement du modèle A

Epoch 1, Loss: 1.3890
Test set: Avg. loss: 1.1502, Accuracy: 2611/5000 (52.22%)
Epoch 2, Loss: 1.0791
Test set: Avg. loss: 0.9640, Accuracy: 2975/5000 (59.50%)
Test set: Avg. loss: 0.9640, Accuracy: 2975/5000 (59.50%)
Epoch 1, Loss: 1.6342
Test set: Avg. loss: 1.6131, Accuracy: 1000/5000 (20.00%)
Epoch 2, Loss: 1.4119
Test set: Avg. loss: 1.2526, Accuracy: 2237/5000 (44.74%)
Test set: Avg. loss: 1.2526, Accuracy: 2237/5000 (44.74%)
Epoch 1, Loss: 1.3922
Test set: Avg. loss: 1.1362, Accuracy: 2501/5000 (50.02%)
Epoch 2, Loss: 1.1100
Test set: Avg. loss: 1.0200, Accuracy: 2942/5000 (58.84%)
Test set: Avg. loss: 1.0200, Accuracy: 2942/5000 (58.84%)
Epoch 1, Loss: 1.6360
Test set: Avg. loss: 1.6171, Accuracy: 1000/5000 (20.00%)
Epoch 2, Loss: 1.2982
Test set: Avg. loss: 1.1269, Accuracy: 2546/5000 (50.92%)
Test set: Avg. loss: 1.1269, Accuracy: 2546/5000 (50.92%)
Epoch 1, Loss: 1.3694
Test set: Avg. loss: 1.1274, Accuracy: 2604/5000 (52.08%)


KeyboardInterrupt: 